In [3]:
import pandas as pd
import tkinter as tk
from tkinter import ttk, messagebox
import os
from pathlib import Path
from datetime import datetime as _dt
from datetime import datetime

# -----------------------
# Initialize DataFrames
# -----------------------
case_rep_df = pd.DataFrame(columns=['Case Rep Name', 'CR Email', 'CR Phone', 'CR Fax'])

physicians_df = pd.DataFrame(columns=['Physcian Name', 'Speciality', 'CA License No.', 'Physician Email',
                                      'Client Rep Email', 'Biller Email', 'Accounting Email', 'Named Email Sender'])

service_items_df = pd.DataFrame(columns=['Item Number', 'CPT Code & Mod', 'Entity Type', 'Item Description',
                                         'Full Description', 'Anes Units', 'Anes Rate', 'Item Charge'])

pi_attorney_df = pd.DataFrame(columns=['Attorney Name', 'Firm', 'Address', 'City', 'State', 'Zip',
                                       'Work Phone', 'Mobile Phone', 'Assistant Name', 'Email', 'Asst Phone'])

pi_patients_df = pd.DataFrame(columns=['Patient ID', 'Patient Fname', 'Patient Lname', 'DOB', 'Patient Phone',
                                       'Address', 'City', 'State', 'Zip', 'Physcians', 'Case Reps',
                                       'Attorneys', 'DOI', 'Patient Email'])

# Open Cases DataFrame
open_cases_df = pd.DataFrame(columns=['First Name', 'Last Name', 'Service Type', 'Physcian Name', 'Date of Service', 
                                     'Location Name', 'Date Billed', 'Service Provided', 'Full Service Description', 
                                     'Law Firm', 'Attorney', 'Case Rep', 'Item Charge', 'Pmt Date', 'Paid Amount', 
                                     'Collection %', 'Lien Reduction %', 'Settlement Accepted', 'Discount Date', 'Discount Amount'])

# Location Clinic DataFrame
location_clinic_df = pd.DataFrame(columns=['Location Name', 'Address', 'City', 'State', 'Zip', 'Phone', 'Fax'])

# -----------------------
# Load existing data from files
# -----------------------
def _get_base_dir():
    """Resolve base directory that contains 'DataBase' folder.
    Tries current working dir, project paths, and known user locations on this machine.
    """
    candidates = [
        Path.cwd(),
        Path.cwd().parent,
        Path(r'C:\Users\Mazen\Downloads\Old DataBase 2\Old DataBase'),
        Path('/Users/mazen/Desktop/Old Data Base'),
        Path('/Users/mazen/Desktop/Old DataBase'),
    ]
    for base in candidates:
        try:
            if (base / 'DataBase').exists():
                return base
        except Exception:
            pass
    # Fallback to cwd
    return Path.cwd()

def load_initial_data():
    global case_rep_df, physicians_df, service_items_df, pi_attorney_df, pi_patients_df, open_cases_df, location_clinic_df
    base = _get_base_dir() / 'DataBase'
    
    # Load each file if it exists
    files_to_load = [
        ('Case Rep.csv', case_rep_df, 'Case Rep Name'),
        ('Physicians.csv', physicians_df, 'Physcian Name'),
        ('PI Attorney.csv', pi_attorney_df, 'Attorney Name'),
        ('PI Patients.csv', pi_patients_df, 'Patient Fname'),
        ('Service Items.csv', service_items_df, 'Item Number'),
        ('Transactions - Open Case AR Report.csv', open_cases_df, 'First Name'),
        ('Location  Clinic.csv', location_clinic_df, 'Location Name'),
    ]
    
    print(f"Loading data from: {base}")
    
    for fname, target_df, key_col in files_to_load:
        path = base / fname
        if path.exists():
            try:
                loaded = pd.read_csv(path)
            except Exception:
                try:
                    loaded = pd.read_excel(path)
                except Exception:
                    print(f"Could not load {fname}")
                    continue
            
            # Align columns with target schema
            if not target_df.empty:
                wanted_cols = list(target_df.columns)
                for c in loaded.columns:
                    if c not in wanted_cols:
                        wanted_cols.append(c)
                loaded = loaded.reindex(columns=wanted_cols)
            
            # Assign to global variable
            if fname == 'Case Rep.csv':
                case_rep_df = loaded
                print(f"Loaded {len(case_rep_df)} Case Reps")
            elif fname == 'Physicians.csv':
                physicians_df = loaded
                print(f"Loaded {len(physicians_df)} Physicians")
            elif fname == 'PI Attorney.csv':
                pi_attorney_df = loaded
                print(f"Loaded {len(pi_attorney_df)} Attorneys")
            elif fname == 'PI Patients.csv':
                pi_patients_df = loaded
                ensure_patient_ids()
                print(f"Loaded {len(pi_patients_df)} Patients")
            elif fname == 'Service Items.csv':
                service_items_df = loaded
                print(f"Loaded {len(service_items_df)} Service Items")
            elif fname == 'Transactions - Open Case AR Report.csv':
                open_cases_df = loaded
                print(f"Loaded {len(open_cases_df)} Open Cases")
            elif fname == 'Location  Clinic.csv':
                location_clinic_df = loaded
                print(f"Loaded {len(location_clinic_df)} Locations")
                print(f"Location columns: {list(location_clinic_df.columns)}")
                if not location_clinic_df.empty:
                    print(f"Sample locations: {location_clinic_df['Location Name'].head().tolist()}")

# Load on startup (deferred below)
# Will call load_initial_data() after utility function definitions

# -----------------------
# Utility functions
# -----------------------
def safe_input(prompt, default=""):
    """Input with default fallback (console fallback used only if GUI not used)"""
    val = input(prompt)
    return val.strip() if val.strip() else default

def generate_patient_id():
    """Generate next unique Patient ID like P001, P002 based on existing IDs."""
    global pi_patients_df
    existing_ids = []
    if 'Patient ID' in pi_patients_df.columns:
        existing_ids = [str(x) for x in pi_patients_df['Patient ID'].dropna().astype(str).tolist()]
    max_num = 0
    for pid in existing_ids:
        if pid.startswith('P') and pid[1:].isdigit():
            max_num = max(max_num, int(pid[1:]))
    next_num = max_num + 1
    return f"P{next_num:03d}"

def ensure_patient_ids():
    """Ensure `Patient ID` column exists and all rows have IDs. Preserve existing IDs."""
    global pi_patients_df
    if 'Patient ID' not in pi_patients_df.columns:
        pi_patients_df.insert(0, 'Patient ID', None)
    # Fill missing IDs
    for idx, val in pi_patients_df['Patient ID'].items():
        if not val or str(val).strip() == "":
            pi_patients_df.at[idx, 'Patient ID'] = generate_patient_id()


def check_duplicate_patient(fname, lname, dob):
    """Check if patient already exists by name and DOB"""
    global pi_patients_df
    if pi_patients_df.empty:
        return False
    
    fname_clean = str(fname).strip().lower()
    lname_clean = str(lname).strip().lower()
    dob_clean = normalize_date(str(dob).strip())
    
    for _, row in pi_patients_df.iterrows():
        existing_fname = str(row.get('Patient Fname', '')).strip().lower()
        existing_lname = str(row.get('Patient Lname', '')).strip().lower()
        existing_dob = normalize_date(str(row.get('DOB', '')))
        
        if (fname_clean == existing_fname and 
            lname_clean == existing_lname and 
            dob_clean == existing_dob):
            return True
    return False


def check_duplicate_attorney(name, firm):
    """Check if attorney already exists by name and firm"""
    global pi_attorney_df
    if pi_attorney_df.empty:
        return False
    
    name_clean = str(name).strip().lower()
    firm_clean = str(firm).strip().lower()
    
    for _, row in pi_attorney_df.iterrows():
        existing_name = str(row.get('Attorney Name', '')).strip().lower()
        existing_firm = str(row.get('Firm', '')).strip().lower()
        
        if name_clean == existing_name and firm_clean == existing_firm:
            return True
    return False


def check_duplicate_case_rep(name):
    """Check if case rep already exists by name"""
    global case_rep_df
    if case_rep_df.empty:
        return False
    
    name_clean = str(name).strip().lower()
    
    for _, row in case_rep_df.iterrows():
        existing_name = str(row.get('Case Rep Name', '')).strip().lower()
        
        if name_clean == existing_name:
            return True
    return False

# Date normalization helper
def normalize_date(date_str: str) -> str:
    s = (date_str or "").strip()
    if not s:
        return ""
    for fmt in ("%m/%d/%Y", "%Y-%m-%d", "%m-%d-%Y", "%Y/%m/%d", "%d-%m-%Y", "%d/%m/%Y"):
        try:
            dt = datetime.strptime(s, fmt)
            return dt.strftime("%m/%d/%Y")
        except ValueError:
            continue
    # If parsing fails, return original so user can correct
    return s

# Now that helpers are defined, load initial data
load_initial_data()

# -----------------------
# Core Add Functions (logic only)
# -----------------------
def add_case_rep_record(name, email, phone, fax):
    new_row = {
        'Case Rep Name': name,
        'CR Email': email,
        'CR Phone': phone,
        'CR Fax': fax
    }
    case_rep_df.loc[len(case_rep_df)] = new_row

def add_physician_record(name, speciality, license_no, email):
    new_row = {
        'Physcian Name': name,
        'Speciality': speciality,
        'CA License No.': license_no,
        'Physician Email': email,
        'Client Rep Email': "",
        'Biller Email': "",
        'Accounting Email': "",
        'Named Email Sender': ""
    }
    physicians_df.loc[len(physicians_df)] = new_row

def add_attorney_record(name, firm, addr, city, state, zip_code, work_phone, mobile, asst_name, email, asst_phone):
    new_row = {
        'Attorney Name': name,
        'Firm': firm,
        'Address': addr,
        'City': city,
        'State': state,
        'Zip': zip_code,
        'Work Phone': work_phone,
        'Mobile Phone': mobile,
        'Assistant Name': asst_name,
        'Email': email,
        'Asst Phone': asst_phone
    }
    pi_attorney_df.loc[len(pi_attorney_df)] = new_row

def add_service_item_record(item_no, cpt, entity, desc, full_desc, anes_units, anes_rate, charge):
    new_row = {
        'Item Number': item_no,
        'CPT Code & Mod': cpt,
        'Entity Type': entity,
        'Item Description': desc,
        'Full Description': full_desc,
        'Anes Units': anes_units,
        'Anes Rate': anes_rate,
        'Item Charge': charge
    }
    service_items_df.loc[len(service_items_df)] = new_row

def add_patient_record(fname, lname, dob, phone, addr, city, state, zip_code, physician, case_rep, attorney, doi, email):
    # Ensure Patient ID column exists
    ensure_patient_ids()
    # Generate new ID
    patient_id = generate_patient_id()
    # Create a new row as a dictionary including Patient ID
    new_row = {
        'Patient ID': patient_id,
        'Patient Fname': fname,
        'Patient Lname': lname,
        'DOB': normalize_date(dob),
        'Patient Phone': phone,
        'Address': addr,
        'City': city,
        'State': state,
        'Zip': zip_code,
        'Physcians': physician,
        'Case Reps': case_rep,
        'Attorneys': attorney,
        'DOI': normalize_date(doi),
        'Patient Email': email
    }
    pi_patients_df.loc[len(pi_patients_df)] = new_row
    return patient_id

def add_open_case_record(first_name, last_name, service_type, physician_name, date_of_service, location_name, 
                        date_billed, service_provided, full_service_desc, law_firm, attorney, case_rep, 
                        item_charge, pmt_date, paid_amount, collection_pct, lien_reduction_pct, 
                        settlement_accepted, discount_date, discount_amount):
    open_cases_df.loc[len(open_cases_df)] = [
        first_name, last_name, service_type, physician_name, normalize_date(date_of_service), 
        location_name, normalize_date(date_billed), service_provided, full_service_desc, 
        law_firm, attorney, case_rep, item_charge, normalize_date(pmt_date), paid_amount, 
        collection_pct, lien_reduction_pct, settlement_accepted, normalize_date(discount_date), discount_amount
    ]

# -----------------------
# GUI Helpers
# -----------------------
class SearchableCombobox(ttk.Combobox):
    def __init__(self, master=None, values=None, textvariable=None, **kwargs):
        super().__init__(master, textvariable=textvariable, state="normal", **kwargs)
        self._all_values = [str(v) for v in (values or [])]
        self.configure(values=self._all_values)
        self.bind('<KeyRelease>', self._on_key_release)
        self.bind('<FocusIn>', self._on_focus_in)

    def set_values(self, values):
        self._all_values = [str(v) for v in (values or [])]
        self.configure(values=self._all_values)

    def _on_key_release(self, event):
        typed = self.get().lower()
        if not typed:
            self.configure(values=self._all_values)
            return
        filtered = [v for v in self._all_values if typed in v.lower()]
        self.configure(values=filtered)
        # Show dropdown if there are filtered results
        if filtered:
            self.event_generate('<Button-1>')

    def _on_focus_in(self, event):
        # Show all values when focused
        self.configure(values=self._all_values)


def open_form(title, fields, on_submit):
    """Create a simple modal form. fields: list of (label, tk.StringVar)"""
    win = tk.Toplevel(root)
    win.title(title)
    win.grab_set()

    container = ttk.Frame(win, padding=12)
    container.grid(row=0, column=0, sticky="nsew")

    for idx, (label, var) in enumerate(fields):
        ttk.Label(container, text=label).grid(row=idx, column=0, sticky="w", padx=4, pady=4)
        ttk.Entry(container, textvariable=var, width=40).grid(row=idx, column=1, sticky="ew", padx=4, pady=4)

    def handle_submit():
        try:
            on_submit()
            win.destroy()
            refresh_preview()
            # Refresh all open forms' dropdowns
            refresh_all_dropdowns()
        except Exception as e:
            messagebox.showerror("Error", str(e))

    btns = ttk.Frame(container)
    btns.grid(row=len(fields), column=0, columnspan=2, pady=(8,0))
    ttk.Button(btns, text="Submit", command=handle_submit).pack(side=tk.LEFT, padx=4)
    ttk.Button(btns, text="Cancel", command=win.destroy).pack(side=tk.LEFT, padx=4)

    win.bind('<Return>', lambda _e: handle_submit())
    win.bind('<Escape>', lambda _e: win.destroy())

# Global variables to track open forms
open_forms = []

def refresh_all_dropdowns():
    """Refresh all open forms' dropdowns"""
    for form_refresh_func in open_forms:
        try:
            form_refresh_func()
        except:
            pass

def refresh_preview():
    """Refresh the preview notebook treeviews from DataFrames"""
    for tree, df in [
        (patients_tree, pi_patients_df),
        (physicians_tree, physicians_df),
        (case_reps_tree, case_rep_df),
        (attorneys_tree, pi_attorney_df),
        (services_tree, service_items_df),
        (open_cases_tree, open_cases_df),
    ]:
        # Clear
        for item_id in tree.get_children():
            tree.delete(item_id)
        # Setup columns if empty
        cols = list(df.columns)
        tree["columns"] = cols
        tree["show"] = "headings"
        for c in cols:
            tree.heading(c, text=c)
            tree.column(c, width=max(100, int(800/len(cols))), stretch=True)
        # Insert last up to 50
        tail_df = df.tail(50)
        for _idx, row in tail_df.iterrows():
            tree.insert("", tk.END, values=[row.get(c, "") for c in cols])

# -----------------------
# GUI Actions
# -----------------------
def gui_add_case_rep():
    name = tk.StringVar()
    email = tk.StringVar()
    phone = tk.StringVar()
    fax = tk.StringVar()

    def submit():
        if not name.get().strip():
            raise ValueError("Case Rep Name is required")
        
        # Check for duplicate case rep
        if check_duplicate_case_rep(name.get().strip()):
            if not messagebox.askyesno("Duplicate Case Rep", 
                                     f"Case Rep {name.get().strip()} already exists.\n\nDo you want to add them anyway?"):
                return
        
        add_case_rep_record(name.get().strip(), email.get().strip(), phone.get().strip(), fax.get().strip())
        messagebox.showinfo("Success", f"Case Rep {name.get().strip()} added successfully!")

    open_form("Add Case Rep", [("Case Rep Name *", name), ("CR Email", email), ("CR Phone", phone), ("CR Fax (Optional)", fax)], submit)

def gui_add_physician():
    name = tk.StringVar()
    speciality = tk.StringVar()
    license_no = tk.StringVar()
    email = tk.StringVar()

    def submit():
        if not name.get().strip():
            raise ValueError("Physician Name is required")
        add_physician_record(name.get().strip(), speciality.get().strip(), license_no.get().strip(), email.get().strip())
        messagebox.showinfo("Success", f"Physician {name.get().strip()} added successfully!")

    open_form("Add Physician", [("Physician Name", name), ("Speciality", speciality), ("CA License No.", license_no), ("Physician Email", email)], submit)

def gui_add_attorney():
    name = tk.StringVar()
    firm = tk.StringVar()
    addr = tk.StringVar()
    city = tk.StringVar()
    state = tk.StringVar()
    zip_code = tk.StringVar()
    work_phone = tk.StringVar()
    mobile = tk.StringVar()
    asst_name = tk.StringVar()
    email = tk.StringVar()
    asst_phone = tk.StringVar()

    def submit():
        if not name.get().strip():
            raise ValueError("Attorney Name is required")
        
        # Check for duplicate attorney
        if check_duplicate_attorney(name.get().strip(), firm.get().strip()):
            if not messagebox.askyesno("Duplicate Attorney", 
                                     f"Attorney {name.get().strip()} at {firm.get().strip()} already exists.\n\nDo you want to add them anyway?"):
                return
        
        add_attorney_record(name.get().strip(), firm.get().strip(), addr.get().strip(), city.get().strip(), state.get().strip(), zip_code.get().strip(), work_phone.get().strip(), mobile.get().strip(), asst_name.get().strip(), email.get().strip(), asst_phone.get().strip())
        messagebox.showinfo("Success", f"Attorney {name.get().strip()} added successfully!")

    open_form(
        "Add Attorney",
        [("Attorney Name", name), ("Firm", firm), ("Address", addr), ("City", city), ("State", state), ("Zip", zip_code), ("Work Phone", work_phone), ("Mobile Phone", mobile), ("Assistant Name", asst_name), ("Email", email), ("Asst Phone", asst_phone)],
        submit,
    )

def gui_add_service_item():
    item_no = tk.StringVar()
    cpt = tk.StringVar()
    entity = tk.StringVar()
    desc = tk.StringVar()
    full_desc = tk.StringVar()
    anes_units = tk.StringVar()
    anes_rate = tk.StringVar()
    charge = tk.StringVar()

    def submit():
        if not item_no.get().strip():
            raise ValueError("Item Number is required")
        add_service_item_record(item_no.get().strip(), cpt.get().strip(), entity.get().strip(), desc.get().strip(), full_desc.get().strip(), anes_units.get().strip(), anes_rate.get().strip(), charge.get().strip())
        messagebox.showinfo("Success", f"Service Item {item_no.get().strip()} added successfully!")

    open_form(
        "Add Service Item",
        [("Item Number", item_no), ("CPT Code & Mod", cpt), ("Entity Type", entity), ("Item Description", desc), ("Full Description", full_desc), ("Anes Units", anes_units), ("Anes Rate", anes_rate), ("Item Charge", charge)],
        submit,
    )

def gui_add_patient():
    fname = tk.StringVar()
    lname = tk.StringVar()
    dob = tk.StringVar()
    phone = tk.StringVar()
    addr = tk.StringVar()
    city = tk.StringVar()
    state = tk.StringVar()
    zip_code = tk.StringVar()
    doi = tk.StringVar()
    email = tk.StringVar()

    phys_choice = tk.StringVar()
    rep_choice = tk.StringVar()
    att_choice = tk.StringVar()

    def submit():
        if not fname.get().strip() or not lname.get().strip():
            raise ValueError("First and Last Name are required")
        
        # Check for duplicate patient
        if check_duplicate_patient(fname.get().strip(), lname.get().strip(), dob.get().strip()):
            if not messagebox.askyesno("Duplicate Patient", 
                                     f"Patient {fname.get().strip()} {lname.get().strip()} with DOB {dob.get().strip()} already exists.\n\nDo you want to add them anyway?"):
                return
        
        # Create new patient
        new_id = add_patient_record(
            fname.get().strip(), lname.get().strip(), dob.get().strip(), phone.get().strip(), addr.get().strip(),
            city.get().strip(), state.get().strip(), zip_code.get().strip(), phys_choice.get().strip(), rep_choice.get().strip(), att_choice.get().strip(),
            doi.get().strip(), email.get().strip()  # Optional field
        )
        messagebox.showinfo("Patient Added", f"Patient added with ID {new_id}")

    # Build a custom form with comboboxes and Add New buttons
    win = tk.Toplevel(root)
    win.title("Add New Patient")
    win.grab_set()

    container = ttk.Frame(win, padding=12)
    container.grid(row=0, column=0, sticky="nsew")

    # Input fields
    entries = [
        ("First Name *", fname), ("Last Name *", lname), ("DOB (MM/DD/YYYY)", dob), ("Phone", phone),
        ("Address", addr), ("City", city), ("State", state), ("Zip", zip_code), ("DOI (MM/DD/YYYY)", doi), ("Patient Email (Optional)", email)
    ]
    for idx, (label, var) in enumerate(entries, start=1):
        ttk.Label(container, text=label).grid(row=idx, column=0, sticky="w", padx=4, pady=4)
        ttk.Entry(container, textvariable=var, width=40).grid(row=idx, column=1, sticky="ew", padx=4, pady=4)

    # Combobox rows
    def refresh_choices():
        # Force refresh of all global DataFrames to get latest data
        global case_rep_df, physicians_df, service_items_df, pi_attorney_df, pi_patients_df, open_cases_df, location_clinic_df
        
        # Get unique, sorted names from DataFrames (no placeholders)
        phys_values = sorted([str(x) for x in physicians_df['Physcian Name'].dropna().unique().tolist()]) if not physicians_df.empty else []
        rep_values = sorted([str(x) for x in case_rep_df['Case Rep Name'].dropna().unique().tolist()]) if not case_rep_df.empty else []
        att_values = sorted([str(x) for x in pi_attorney_df['Attorney Name'].dropna().unique().tolist()]) if not pi_attorney_df.empty else []
        
        phys_combo.set_values(phys_values)
        rep_combo.set_values(rep_values)
        att_combo.set_values(att_values)
        
        # Clear selections if current value not in list
        if phys_choice.get() not in phys_values:
            phys_choice.set("")
        if rep_choice.get() not in rep_values:
            rep_choice.set("")
        if att_choice.get() not in att_values:
            att_choice.set("")

    row_base = len(entries) + 1

    ttk.Label(container, text="Physician").grid(row=row_base+0, column=0, sticky="w", padx=4, pady=4)
    phys_combo = SearchableCombobox(container, textvariable=phys_choice, width=37)
    phys_combo.grid(row=row_base+0, column=1, sticky="ew", padx=4, pady=4)
    
    def add_phys_and_refresh():
        gui_add_physician()
        # Force refresh after a short delay to ensure the add operation completes
        root.after(100, refresh_choices)
    
    ttk.Button(container, text="Add New", command=add_phys_and_refresh).grid(row=row_base+0, column=2, padx=4)

    ttk.Label(container, text="Case Rep").grid(row=row_base+1, column=0, sticky="w", padx=4, pady=4)
    rep_combo = SearchableCombobox(container, textvariable=rep_choice, width=37)
    rep_combo.grid(row=row_base+1, column=1, sticky="ew", padx=4, pady=4)
    
    def add_rep_and_refresh():
        gui_add_case_rep()
        # Force refresh after a short delay to ensure the add operation completes
        root.after(100, refresh_choices)
    
    ttk.Button(container, text="Add New", command=add_rep_and_refresh).grid(row=row_base+1, column=2, padx=4)

    ttk.Label(container, text="Attorney").grid(row=row_base+2, column=0, sticky="w", padx=4, pady=4)
    att_combo = SearchableCombobox(container, textvariable=att_choice, width=37)
    att_combo.grid(row=row_base+2, column=1, sticky="ew", padx=4, pady=4)
    
    def add_att_and_refresh():
        gui_add_attorney()
        # Force refresh after a short delay to ensure the add operation completes
        root.after(100, refresh_choices)
    
    ttk.Button(container, text="Add New", command=add_att_and_refresh).grid(row=row_base+2, column=2, padx=4)

    # Buttons
    def handle_submit():
        try:
            submit()
            win.destroy()
            refresh_preview()
            # Refresh all open forms' dropdowns
            refresh_all_dropdowns()
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def cleanup():
        # Remove this form's refresh function from global list
        if refresh_choices in open_forms:
            open_forms.remove(refresh_choices)

    btns = ttk.Frame(container)
    btns.grid(row=row_base+3, column=0, columnspan=3, pady=(8,0))
    ttk.Button(btns, text="Submit", command=handle_submit).pack(side=tk.LEFT, padx=4)
    ttk.Button(btns, text="Cancel", command=lambda: (cleanup(), win.destroy())).pack(side=tk.LEFT, padx=4)

    win.bind('<Return>', lambda _e: handle_submit())
    win.bind('<Escape>', lambda _e: (cleanup(), win.destroy()))
    win.protocol("WM_DELETE_WINDOW", lambda: (cleanup(), win.destroy()))

    # Register this form's refresh function
    open_forms.append(refresh_choices)

    refresh_choices()

def gui_add_open_case():
    first_name = tk.StringVar()
    last_name = tk.StringVar()
    service_type = tk.StringVar()
    physician_name = tk.StringVar()
    date_of_service = tk.StringVar()
    location_name = tk.StringVar()
    date_billed = tk.StringVar()
    date_billed.set(_dt.now().strftime('%m/%d/%Y'))
    service_provided = tk.StringVar()
    full_service_desc = tk.StringVar()
    law_firm = tk.StringVar()
    attorney = tk.StringVar()
    case_rep = tk.StringVar()
    item_charge = tk.StringVar()
    pmt_date = tk.StringVar()
    paid_amount = tk.StringVar()
    collection_pct = tk.StringVar()
    lien_reduction_pct = tk.StringVar()
    settlement_accepted = tk.StringVar()
    discount_date = tk.StringVar()
    discount_amount = tk.StringVar()
    cpt_code = tk.StringVar()

    patient_choice = tk.StringVar()
    patient_display_to_index = {}
    cpt_display_to_index = {}

    def load_selected_patient():
        sel = patient_choice.get()
        print(f"DEBUG: Selected patient: '{sel}'")
        print(f"DEBUG: Available keys: {list(patient_display_to_index.keys())[:3]}...")  # Show first 3 keys
        if sel in patient_display_to_index:
            idx = patient_display_to_index[sel]
            row = pi_patients_df.loc[idx]
            first_name.set(str(row.get('Patient Fname', '')))
            last_name.set(str(row.get('Patient Lname', '')))
            physician_name.set(str(row.get('Physcians', '')))
            case_rep.set(str(row.get('Case Reps', '')))
            attorney.set(str(row.get('Attorneys', '')))
            # Auto-fill firm from selected attorney (if found)
            try:
                selected_att = attorney.get().strip()
                if selected_att:
                    match = pi_attorney_df[pi_attorney_df['Attorney Name'] == selected_att]
                    if not match.empty:
                        law_firm.set(str(match.iloc[0].get('Firm', '')))
            except Exception:
                pass
            # You could also auto-fill other fields if they exist in patient data
            messagebox.showinfo("Patient Loaded", f"Patient {first_name.get()} {last_name.get()} loaded successfully!")
        else:
            messagebox.showwarning("No Selection", "Please select a patient first")

    def submit():
        if not first_name.get().strip() or not last_name.get().strip():
            raise ValueError("First and Last Name are required")
        add_open_case_record(
            first_name.get().strip(), last_name.get().strip(), service_type.get().strip(),
            physician_name.get().strip(), date_of_service.get().strip(), location_name.get().strip(),
            date_billed.get().strip(), service_provided.get().strip(), full_service_desc.get().strip(),
            law_firm.get().strip(), attorney.get().strip(), case_rep.get().strip(),
            item_charge.get().strip(), pmt_date.get().strip(), paid_amount.get().strip(),
            collection_pct.get().strip(), lien_reduction_pct.get().strip(), settlement_accepted.get().strip(),
            discount_date.get().strip(), discount_amount.get().strip()
        )
        messagebox.showinfo("Success", f"Open Case for {first_name.get().strip()} {last_name.get().strip()} added successfully!")

    # Build a custom form with dropdowns
    win = tk.Toplevel(root)
    win.title("Add Open Case")
    win.grab_set()

    container = ttk.Frame(win, padding=12)
    container.grid(row=0, column=0, sticky="nsew")

    # Patient selector
    ttk.Label(container, text="Select Patient").grid(row=0, column=0, sticky="w", padx=4, pady=(0,4))
    patient_combo = SearchableCombobox(container, textvariable=patient_choice, width=37)
    patient_combo.grid(row=0, column=1, sticky="ew", padx=4, pady=(0,4))
    ttk.Button(container, text="Load Patient", command=lambda: (load_selected_patient(), refresh_choices())).grid(row=0, column=2, padx=4, pady=(0,4))

    # Basic fields (only the ones without dropdowns)
    basic_fields = [
        ("First Name *", first_name), ("Last Name *", last_name),
        ("Date of Service (MM/DD/YYYY)", date_of_service),
        ("Date Billed (MM/DD/YYYY)", date_billed),
        ("Payment Date (MM/DD/YYYY) (Optional)", pmt_date), ("Paid Amount (Optional)", paid_amount),
        ("Collection % (Optional)", collection_pct), ("Lien Reduction % (Optional)", lien_reduction_pct),
        ("Settlement Accepted (Optional)", settlement_accepted), ("Discount Date (MM/DD/YYYY) (Optional)", discount_date),
        ("Discount Amount (Optional)", discount_amount)
    ]
    
    for idx, (label, var) in enumerate(basic_fields, start=1):
        ttk.Label(container, text=label).grid(row=idx, column=0, sticky="w", padx=4, pady=4)
        ttk.Entry(container, textvariable=var, width=40).grid(row=idx, column=1, sticky="ew", padx=4, pady=4)

    def refresh_choices():
        # Force refresh of all global DataFrames to get latest data
        global case_rep_df, physicians_df, service_items_df, pi_attorney_df, pi_patients_df, open_cases_df, location_clinic_df
        
        # Physician dropdown
        phys_values = sorted([str(x) for x in physicians_df['Physcian Name'].dropna().unique().tolist()]) if not physicians_df.empty else []
        phys_combo.set_values(phys_values)
        if physician_name.get() not in phys_values:
            physician_name.set("")

        # Case Rep dropdown
        rep_values = sorted([str(x) for x in case_rep_df['Case Rep Name'].dropna().unique().tolist()]) if not case_rep_df.empty else []
        rep_combo.set_values(rep_values)
        if case_rep.get() not in rep_values:
            case_rep.set("")

        # Attorney dropdown
        att_values = sorted([str(x) for x in pi_attorney_df['Attorney Name'].dropna().unique().tolist()]) if not pi_attorney_df.empty else []
        att_combo.set_values(att_values)
        if attorney.get() not in att_values:
            attorney.set("")
        else:
            # Auto-fill firm if attorney is selected
            on_attorney_change()

        # Law Firm dropdown (from attorney data)
        firm_values = sorted([str(x) for x in pi_attorney_df['Firm'].dropna().unique().tolist()]) if not pi_attorney_df.empty else []
        firm_combo.set_values(firm_values)

        # CPT Code dropdown (composite label: "CPT Code & Mod | Entity Type")
        cpt_display_to_index.clear()
        if not service_items_df.empty and 'CPT Code & Mod' in service_items_df.columns:
            cpt_values = []
            for idx, row in service_items_df.iterrows():
                code = str(row.get('CPT Code & Mod', '')).strip()
                ent = str(row.get('Entity Type', '')).strip()
                label = f"{code} | {ent}" if code or ent else ""
                if label:
                    cpt_display_to_index[label] = idx
                    cpt_values.append(label)
            cpt_values = sorted(list(dict.fromkeys(cpt_values)))
            cpt_combo.set_values(cpt_values)
            if cpt_code.get() not in cpt_values:
                cpt_code.set("")
        else:
            cpt_combo.set_values([])
            cpt_code.set("")

        # Service Type dropdown (Entity Type from Service Items)
        service_type_values = sorted([str(x) for x in service_items_df['Entity Type'].dropna().unique().tolist()]) if not service_items_df.empty else []
        service_type_combo.set_values(service_type_values)
        if service_type.get() not in service_type_values:
            service_type.set("")

        # Service Provided dropdown (Item Description from Service Items)
        service_provided_values = sorted([str(x) for x in service_items_df['Item Description'].dropna().unique().tolist()]) if not service_items_df.empty else []
        service_provided_combo.set_values(service_provided_values)
        if service_provided.get() not in service_provided_values:
            service_provided.set("")

        # Location Name dropdown (from Location Clinic)
        location_values = sorted([str(x) for x in location_clinic_df['Location Name'].dropna().unique().tolist()]) if not location_clinic_df.empty else []
        location_combo.set_values(location_values)
        if location_name.get() not in location_values:
            location_name.set("")

    def _compute_time_units(total_minutes: int) -> int:
        if total_minutes <= 0:
            return 0
        return (total_minutes + 14) // 15

    def _set_location_by_service_type():
        st = (service_type.get() or "").strip().lower()
        mapping = {
            'anesthesia': 'ANESTHESIOLOGY & PERIOPERATIVE MEDICINE SPECIALISTS',
            'facility': 'UNIVERSITY SURGICAL INSTITUTE, LLC',
            'professional': 'SCOPES HEALTH INC.'
        }
        if st in mapping:
            location_name.set(mapping[st])

    def _open_anesthesia_dialog():
        dlg = tk.Toplevel(root)
        dlg.title("Anesthesia Time Entry")
        dlg.grab_set()

        frame = ttk.Frame(dlg, padding=12)
        frame.grid(row=0, column=0, sticky="nsew")

        start_var = tk.StringVar()
        stop_var = tk.StringVar()

        ttk.Label(frame, text="Start Time (HH:MM)").grid(row=0, column=0, sticky="w", padx=4, pady=4)
        ttk.Entry(frame, textvariable=start_var, width=20).grid(row=0, column=1, sticky="ew", padx=4, pady=4)

        ttk.Label(frame, text="Stop Time (HH:MM)").grid(row=1, column=0, sticky="w", padx=4, pady=4)
        ttk.Entry(frame, textvariable=stop_var, width=20).grid(row=1, column=1, sticky="ew", padx=4, pady=4)

        def submit_anesthesia():
            try:
                def parse_hhmm(val: str) -> int:
                    s = (val or "").strip()
                    h, m = s.split(":")
                    return int(h) * 60 + int(m)
                start_total = parse_hhmm(start_var.get())
                stop_total = parse_hhmm(stop_var.get())
                total_minutes = max(0, stop_total - start_total)
                time_units = _compute_time_units(total_minutes)
                base_units = 5
                total_units = base_units + time_units
                total_bill = total_units * 315
                item_charge.set(str(total_bill))
                # Optionally annotate details in description
                desc = f"Anesthesia: {total_minutes} mins | Base {base_units} + Time {time_units} = {total_units} units @ $315"
                if not full_service_desc.get().strip():
                    full_service_desc.set(desc)
                messagebox.showinfo("Calculated", f"Minutes: {total_minutes}\nTime Units: {time_units}\nTotal Units: {total_units}\nCharge: ${total_bill}")
                dlg.destroy()
            except Exception:
                messagebox.showerror("Invalid Input", "Please enter times in HH:MM format, e.g., 09:05 and 10:20.")

        btns = ttk.Frame(frame)
        btns.grid(row=2, column=0, columnspan=2, pady=(8,0))
        ttk.Button(btns, text="Calculate", command=submit_anesthesia).pack(side=tk.LEFT, padx=4)
        ttk.Button(btns, text="Cancel", command=dlg.destroy).pack(side=tk.LEFT, padx=4)

        dlg.bind('<Return>', lambda _e: submit_anesthesia())
        dlg.bind('<Escape>', lambda _e: dlg.destroy())

    def on_service_provided_change(*args):
        # When service provided changes, auto-fill related fields
        selected_service = service_provided.get()
        if selected_service and selected_service.strip():
            # Find matching service item
            matching_items = service_items_df[service_items_df['Item Description'] == selected_service]
            if not matching_items.empty:
                item = matching_items.iloc[0]
                # Auto-fill related fields
                service_type.set(str(item.get('Entity Type', '')))
                item_charge.set(str(item.get('Item Charge', '')))
                full_service_desc.set(str(item.get('Full Description', '')))
                composite = f"{str(item.get('CPT Code & Mod', '')).strip()} | {str(item.get('Entity Type','')).strip()}".strip()
                cpt_code.set(composite)
                _set_location_by_service_type()
        # If anesthesia service, prompt for times and compute bill
        if "anesthesia" in selected_service.strip().lower():
            _open_anesthesia_dialog()

    def on_cpt_change(*args):
        # When CPT code changes, auto-fill related fields
        selected_cpt_label = cpt_code.get()
        if selected_cpt_label and selected_cpt_label.strip():
            idx = cpt_display_to_index.get(selected_cpt_label)
            if idx is not None and 0 <= idx < len(service_items_df):
                item = service_items_df.iloc[idx]
                service_type.set(str(item.get('Entity Type', '')))
                item_charge.set(str(item.get('Item Charge', '')))
                full_service_desc.set(str(item.get('Full Description', '')))
                service_provided.set(str(item.get('Item Description', '')))
                _set_location_by_service_type()

    def on_service_type_change(*args):
        # When service type changes, auto-fill location
        _set_location_by_service_type()

    def on_attorney_change(*args):
        # When attorney changes, auto-fill law firm
        selected_attorney = attorney.get().strip()
        if selected_attorney:
            matching_attorneys = pi_attorney_df[pi_attorney_df['Attorney Name'] == selected_attorney]
            if not matching_attorneys.empty:
                firm_name = str(matching_attorneys.iloc[0].get('Firm', ''))
                law_firm.set(firm_name)

    def refresh_patient_choices():
        # Force refresh of global patient data
        global pi_patients_df
        patient_display_to_index.clear()
        values = []
        for idx, row in pi_patients_df.iterrows():
            label = f"{str(row.get('Patient Fname','')).strip()} {str(row.get('Patient Lname','')).strip()} | {str(row.get('DOB','')).strip()}"
            patient_display_to_index[label] = idx
            values.append(label)
        patient_combo.set_values(values)
        if not patient_choice.get() and values:
            patient_choice.set("")

    row_base = len(basic_fields) + 1

    # Dropdown fields
    ttk.Label(container, text="Physician").grid(row=row_base+0, column=0, sticky="w", padx=4, pady=4)
    phys_combo = SearchableCombobox(container, textvariable=physician_name, width=37)
    phys_combo.grid(row=row_base+0, column=1, sticky="ew", padx=4, pady=4)
    def add_phys_and_refresh():
        gui_add_physician()
        # Force refresh after a short delay to ensure the add operation completes
        root.after(100, refresh_choices)
    
    ttk.Button(container, text="Add New", command=add_phys_and_refresh).grid(row=row_base+0, column=2, padx=4)

    ttk.Label(container, text="Case Rep").grid(row=row_base+1, column=0, sticky="w", padx=4, pady=4)
    rep_combo = SearchableCombobox(container, textvariable=case_rep, width=37)
    rep_combo.grid(row=row_base+1, column=1, sticky="ew", padx=4, pady=4)
    
    def add_rep_and_refresh():
        gui_add_case_rep()
        # Force refresh after a short delay to ensure the add operation completes
        root.after(100, refresh_choices)
    
    ttk.Button(container, text="Add New", command=add_rep_and_refresh).grid(row=row_base+1, column=2, padx=4)

    ttk.Label(container, text="Attorney").grid(row=row_base+2, column=0, sticky="w", padx=4, pady=4)
    att_combo = SearchableCombobox(container, textvariable=attorney, width=37)
    att_combo.grid(row=row_base+2, column=1, sticky="ew", padx=4, pady=4)
    att_combo.bind('<<ComboboxSelected>>', on_attorney_change)
    
    def add_att_and_refresh():
        gui_add_attorney()
        # Force refresh after a short delay to ensure the add operation completes
        root.after(100, refresh_choices)
    
    ttk.Button(container, text="Add New", command=add_att_and_refresh).grid(row=row_base+2, column=2, padx=4)

    ttk.Label(container, text="Law Firm").grid(row=row_base+3, column=0, sticky="w", padx=4, pady=4)
    firm_combo = SearchableCombobox(container, textvariable=law_firm, width=37)
    firm_combo.grid(row=row_base+3, column=1, sticky="ew", padx=4, pady=4)

    # Service-related dropdowns
    ttk.Label(container, text="CPT Code").grid(row=row_base+4, column=0, sticky="w", padx=4, pady=4)
    cpt_combo = SearchableCombobox(container, textvariable=cpt_code, width=37)
    cpt_combo.grid(row=row_base+4, column=1, sticky="ew", padx=4, pady=4)
    cpt_combo.bind('<<ComboboxSelected>>', on_cpt_change)

    ttk.Label(container, text="Service Type").grid(row=row_base+5, column=0, sticky="w", padx=4, pady=4)
    service_type_combo = SearchableCombobox(container, textvariable=service_type, width=37)
    service_type_combo.grid(row=row_base+5, column=1, sticky="ew", padx=4, pady=4)
    service_type_combo.bind('<<ComboboxSelected>>', on_service_type_change)

    ttk.Label(container, text="Service Provided").grid(row=row_base+6, column=0, sticky="w", padx=4, pady=4)
    service_provided_combo = SearchableCombobox(container, textvariable=service_provided, width=37)
    service_provided_combo.grid(row=row_base+6, column=1, sticky="ew", padx=4, pady=4)
    service_provided_combo.bind('<<ComboboxSelected>>', on_service_provided_change)

    ttk.Label(container, text="Item Charge").grid(row=row_base+7, column=0, sticky="w", padx=4, pady=4)
    item_charge_entry = ttk.Entry(container, textvariable=item_charge, width=40)
    item_charge_entry.grid(row=row_base+7, column=1, sticky="ew", padx=4, pady=4)

    ttk.Label(container, text="Full Service Description").grid(row=row_base+8, column=0, sticky="w", padx=4, pady=4)
    full_service_desc_entry = ttk.Entry(container, textvariable=full_service_desc, width=40)
    full_service_desc_entry.grid(row=row_base+8, column=1, sticky="ew", padx=4, pady=4)

    ttk.Label(container, text="Location Name").grid(row=row_base+9, column=0, sticky="w", padx=4, pady=4)
    location_combo = SearchableCombobox(container, textvariable=location_name, width=37)
    location_combo.grid(row=row_base+9, column=1, sticky="ew", padx=4, pady=4)

    # Buttons
    def handle_submit():
        try:
            submit()
            win.destroy()
            refresh_preview()
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def cleanup():
        # Remove this form's refresh function from global list
        if refresh_choices in open_forms:
            open_forms.remove(refresh_choices)

    btns = ttk.Frame(container)
    btns.grid(row=row_base+10, column=0, columnspan=3, pady=(8,0))
    ttk.Button(btns, text="Submit", command=handle_submit).pack(side=tk.LEFT, padx=4)
    ttk.Button(btns, text="Cancel", command=lambda: (cleanup(), win.destroy())).pack(side=tk.LEFT, padx=4)

    win.bind('<Return>', lambda _e: handle_submit())
    win.bind('<Escape>', lambda _e: (cleanup(), win.destroy()))
    win.protocol("WM_DELETE_WINDOW", lambda: (cleanup(), win.destroy()))

    # Register this form's refresh function
    open_forms.append(refresh_choices)

    refresh_patient_choices()
    refresh_choices()

# -----------------------
# GUI Layout
# -----------------------
root = tk.Tk()
root.title("Data Entry GUI")

main = ttk.Frame(root, padding=12)
main.grid(row=0, column=0, sticky="nsew")
root.rowconfigure(0, weight=1)
root.columnconfigure(0, weight=1)

# Buttons column
btns = ttk.Frame(main)
btns.grid(row=0, column=0, sticky="ns", padx=(0, 12))

# Patient-first primary action
ttk.Button(btns, text="Add Patient", command=gui_add_patient).pack(fill=tk.X, pady=4)

# Open Cases - Most Important
ttk.Button(btns, text="Add Open Case", command=gui_add_open_case).pack(fill=tk.X, pady=4)

# Secondary quick access (optional)
ttk.Button(btns, text="Add Physician", command=gui_add_physician).pack(fill=tk.X, pady=4)
ttk.Button(btns, text="Add Case Rep", command=gui_add_case_rep).pack(fill=tk.X, pady=4)
ttk.Button(btns, text="Add Attorney", command=gui_add_attorney).pack(fill=tk.X, pady=4)
ttk.Button(btns, text="Add Service Item", command=gui_add_service_item).pack(fill=tk.X, pady=4)


# Save/Export functionality
def save_all_dataframes():
    base_dir = Path('D:/Work') if Path('D:/Work').exists() else Path('D:/Work/Old DataBase')
    db_dir = base_dir / 'DataBase'
    out_root = base_dir / 'DataBase SnapShots'
    ts = _dt.now().strftime('%Y%m%d_%H%M%S')
    snapshot_dir = out_root / f'Updated_Data_{ts}'
    snapshot_dir.mkdir(parents=True, exist_ok=True)

    mapping = [
        (case_rep_df, 'Case Rep.csv'),
        (physicians_df, 'Physicians.csv'),
        (pi_attorney_df, 'PI Attorney.csv'),
        (pi_patients_df, 'PI Patients.csv'),
        (service_items_df, 'Service Items.csv'),
        (open_cases_df, 'Transactions - Open Case AR Report.csv'),
        (location_clinic_df, 'Location  Clinic.csv'),
    ]

    for df, fname in mapping:
        orig_path = db_dir / fname
        # Overwrite originals with current in-memory data to avoid duplication
        updated_df = df.copy()

        if orig_path.suffix.lower() in ['.xlsx', '.xls']:
            updated_df.to_excel(orig_path, index=False)
        else:
            updated_df.to_csv(orig_path, index=False)

        # Always write a CSV snapshot copy
        updated_copy_path = snapshot_dir / (orig_path.stem + '.csv')
        updated_df.to_csv(updated_copy_path, index=False)

    messagebox.showinfo('Saved', f'All data saved. Snapshot: {snapshot_dir}')

def reset_memory_and_reload():
    if not messagebox.askyesno("Reload from Files", "Discard unsaved in-memory changes and reload from files?"):
        return
    load_initial_data()
    refresh_preview()
    # Refresh any open forms' dropdowns to reflect reloaded data
    try:
        refresh_all_dropdowns()
    except Exception:
        pass
    messagebox.showinfo("Reloaded", "Data reloaded from files.")


def _dedupe_df(df: pd.DataFrame, subset_cols):
    existing = [c for c in subset_cols if c in df.columns]
    if existing:
        return df.drop_duplicates(subset=existing, keep='first').reset_index(drop=True)
    return df.drop_duplicates(keep='first').reset_index(drop=True)


def dedupe_all_dataframes():
    global case_rep_df, physicians_df, service_items_df, pi_attorney_df, pi_patients_df, open_cases_df, location_clinic_df

    before_counts = {
        'Case Reps': len(case_rep_df),
        'Physicians': len(physicians_df),
        'Attorneys': len(pi_attorney_df),
        'Patients': len(pi_patients_df),
        'Service Items': len(service_items_df),
        'Open Cases': len(open_cases_df),
        'Locations': len(location_clinic_df),
    }

    case_rep_df = _dedupe_df(case_rep_df, ['Case Rep Name'])
    physicians_df = _dedupe_df(physicians_df, ['Physcian Name', 'CA License No.', 'Speciality', 'Physician Email'])
    pi_attorney_df = _dedupe_df(pi_attorney_df, ['Attorney Name', 'Firm'])
    # Patients: prefer Patient ID, else name + DOB
    if 'Patient ID' in pi_patients_df.columns and pi_patients_df['Patient ID'].notna().any():
        pi_patients_df = _dedupe_df(pi_patients_df, ['Patient ID'])
    else:
        pi_patients_df = _dedupe_df(pi_patients_df, ['Patient Fname', 'Patient Lname', 'DOB'])
    service_items_df = _dedupe_df(service_items_df, ['Item Number']) if 'Item Number' in service_items_df.columns else _dedupe_df(service_items_df, ['Item Description', 'CPT Code & Mod', 'Entity Type'])
    open_cases_df = _dedupe_df(open_cases_df, ['First Name', 'Last Name', 'Date of Service', 'Service Provided', 'Location Name', 'Physcian Name'])
    location_clinic_df = _dedupe_df(location_clinic_df, ['Location Name'])

    after_counts = {
        'Case Reps': len(case_rep_df),
        'Physicians': len(physicians_df),
        'Attorneys': len(pi_attorney_df),
        'Patients': len(pi_patients_df),
        'Service Items': len(service_items_df),
        'Open Cases': len(open_cases_df),
        'Locations': len(location_clinic_df),
    }

    refresh_preview()
    try:
        refresh_all_dropdowns()
    except Exception:
        pass

    summary_lines = [f"{k}: {before_counts[k]} -> {after_counts[k]}" for k in before_counts]
    messagebox.showinfo('Deduped', 'Removed duplicates by key.\n' + "\n".join(summary_lines))

# Place buttons after functions are defined
# Action buttons
# Refresh Preview

ttk.Button(btns, text="Refresh Preview", command=lambda: refresh_preview()).pack(fill=tk.X, pady=(12,0))
# Save and Reload

ttk.Button(btns, text="Save (Append + Snapshot)", command=lambda: (save_all_dataframes(), refresh_preview())).pack(fill=tk.X, pady=(12,0))
ttk.Button(btns, text="Reload from Files", command=reset_memory_and_reload).pack(fill=tk.X, pady=(4,0))

ttk.Button(btns, text="Deduplicate All", command=lambda: (dedupe_all_dataframes(),)).pack(fill=tk.X, pady=(12,0))

# Preview notebook
notebook = ttk.Notebook(main)
notebook.grid(row=0, column=1, sticky="nsew")
main.rowconfigure(0, weight=1)
main.columnconfigure(1, weight=1)

patients_frame = ttk.Frame(notebook)
open_cases_frame = ttk.Frame(notebook)
physicians_frame = ttk.Frame(notebook)
case_reps_frame = ttk.Frame(notebook)
attorneys_frame = ttk.Frame(notebook)
services_frame = ttk.Frame(notebook)

notebook.add(patients_frame, text="Patients")
notebook.add(open_cases_frame, text="Open Cases")
notebook.add(physicians_frame, text="Physicians")
notebook.add(case_reps_frame, text="Case Reps")
notebook.add(attorneys_frame, text="Attorneys")
notebook.add(services_frame, text="Service Items")

patients_tree = ttk.Treeview(patients_frame)
open_cases_tree = ttk.Treeview(open_cases_frame)
physicians_tree = ttk.Treeview(physicians_frame)
case_reps_tree = ttk.Treeview(case_reps_frame)
attorneys_tree = ttk.Treeview(attorneys_frame)
services_tree = ttk.Treeview(services_frame)

for tree, frame in [
    (patients_tree, patients_frame),
    (open_cases_tree, open_cases_frame),
    (physicians_tree, physicians_frame),
    (case_reps_tree, case_reps_frame),
    (attorneys_tree, attorneys_frame),
    (services_tree, services_frame),
]:
    tree.pack(fill=tk.BOTH, expand=True)

refresh_preview()

# -----------------------
# Run
# -----------------------
if __name__ == "__main__":
    try:
        root.mainloop()
    except KeyboardInterrupt:
        pass


Loading data from: d:\Work\DataBase
Loaded 77 Case Reps
Loaded 3 Physicians
Loaded 61 Attorneys
Loaded 103 Patients
Loaded 63 Service Items
Loaded 444 Open Cases
Loaded 4 Locations
Location columns: ['Location Name', 'Location Address', 'L-CIty', 'L-State']
Sample locations: ['ANESTHESIOLOGY & PERIOPERATIVE MEDICINE SPECIALISTS', 'SCOPES HEALTH INC.', 'SCOPES HEALTH INC.', 'UNIVERSITY SURGICAL INSTITUTE, LLC']
DEBUG: Selected patient: 'Elvia Rodriguez Franco | 08/26/1960'
DEBUG: Available keys: ['Sharaf Ali | 12/15/1968', 'Oliver Alvarez Barajas | 11/15/2005', 'Mayra Angeles Rodriguez | 6/4/1985']...
DEBUG: Selected patient: 'Jami Highfill | 3/15/1990'
DEBUG: Available keys: ['Sharaf Ali | 12/15/1968', 'Oliver Alvarez Barajas | 11/15/2005', 'Mayra Angeles Rodriguez | 6/4/1985']...
DEBUG: Selected patient: 'Daniel Garcia | 9/26/1972'
DEBUG: Available keys: ['Sharaf Ali | 12/15/1968', 'Oliver Alvarez Barajas | 11/15/2005', 'Mayra Angeles Rodriguez | 6/4/1985']...
DEBUG: Selected patient:

In [4]:
# Windows-stable SearchableCombobox override (debounced, non-intrusive)
try:
    import tkinter as tk
    from tkinter import ttk
except Exception:
    pass

class SearchableCombobox(ttk.Combobox):
    def __init__(self, master=None, values=None, textvariable=None, **kwargs):
        super().__init__(master, textvariable=textvariable, state="normal", **kwargs)
        self._all_values = [str(v) for v in (values or [])]
        self._debounce_after_id = None
        self._is_internal_update = False
        self.configure(values=self._all_values)
        self.bind('<KeyRelease>', self._on_key_release)
        self.bind('<FocusIn>', self._on_focus_in)

    def set_values(self, values):
        # Replace list while preserving current text and avoiding recursion
        self._is_internal_update = True
        try:
            current = self.get()
            self._all_values = [str(v) for v in (values or [])]
            self.configure(values=self._all_values)
            if current:
                self.set(current)
        finally:
            self._is_internal_update = False

    def _apply_filter(self):
        self._debounce_after_id = None
        if self._is_internal_update:
            return
        typed = self.get()
        low = (typed or '').lower()
        self._is_internal_update = True
        try:
            if not low:
                self.configure(values=self._all_values)
                # Do not force-open dropdown on Windows; user can press Down/Alt+Down
                return
            filtered = [v for v in self._all_values if low in v.lower()]
            self.configure(values=filtered)
            # Keep typed text
            self.set(typed)
        finally:
            self._is_internal_update = False

    def _on_key_release(self, event):
        if self._is_internal_update:
            return
        # Debounce rapid key events to avoid flicker/reset on Windows
        try:
            if self._debounce_after_id is not None:
                self.after_cancel(self._debounce_after_id)
        except Exception:
            pass
        self._debounce_after_id = self.after(150, self._apply_filter)

    def _on_focus_in(self, event):
        # Populate full list without forcing dropdown to open
        self.configure(values=self._all_values)

